In [1]:
import glob
import os
import sys

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass


# ==============================================================================
# -- imports -------------------------------------------------------------------
# ==============================================================================


import carla

In [2]:
# Używane biblioteki
import carla
from carla import Transform, Location, Rotation    # Potrzebne do przesunięcia punktu początkowego pojazdu
import random
import cv2                                         # Do wyświetlania obrazu
import math
import numpy as np
import csv
import time
import os
# Potrzebne  do storzenia trasy, w teście "Łosia"
import sys 
sys.path.append('/home/kacper/carla/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [3]:
#połączenie z symulacją 
client = carla.Client('localhost',2000)
world = client.get_world()

In [4]:
# Wybór mapy
client.set_timeout(60)
client.load_world('Town05')

In [5]:
#Wczytanie biblioteki z bluprintami 
bib = world.get_blueprint_library()

In [6]:
#Znalezienie punktów w których można tworzyć pojazdy
spawn_points = world.get_map().get_spawn_points()

In [ ]:
#Stworzenie pojazdu 
pojazd_bib = bib.find('vehicle.mini.cooper_s_2021')
miejsce = spawn_points[8]
miejsce = Transform(Location(x=miejsce.location.x + 302, y=miejsce.location.y + 200, z=miejsce.location.z), miejsce.rotation)

#miejsce = random.choice(spawn_points)
pojazd = world.try_spawn_actor(pojazd_bib, miejsce)
print(miejsce)

In [ ]:
#Pojazdy
#'vehicle.lincoln.mkz_2020'
#'vehicle.mercedes.coupe_2020'
#'vehicle.mercedes.sprinter'
#'vehicle.ford.ambulance'
#'vehicle.'volkswagen.t2_2021'
#'vehicle.mitsubishi.fusorosa'
#'vehicle.tesla.cybertruck'
#'vehicle.ford.mustang'
#'vehicle.tesla.model3'
#vehicle.dodge.charger_2020
#vehicle.nissan.micra
#'vehicle.micro.microlino'
#'vehicle.mini.cooper_s_2021'

# Tworzenie obiektu Location z wartościami x, y, z
miejsce_location = carla.Location(x=35.056522, y=163.656631, z=0.300000)

# Wypisanie wartości zmiennej
print(miejsce_location)

In [8]:
#Przeniesienie do lokazji utworzonego pojazdu
spectator = world.get_spectator()
transform = carla.Transform(pojazd.get_transform().transform(carla.Location(x=-4,z=2.5)),pojazd.get_transform().rotation) 
spectator.set_transform(transform) 

In [ ]:
# Dodanie kamery
pozycja_z = 4 
pozycja_x = -10 

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') 
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=pozycja_z,x=pozycja_x))
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=pojazd)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}

camera.listen(lambda image: camera_callback(image,camera_data))

#Dodanie czujnika IMU

imu_bp = bib.find('sensor.other.imu')
imu_sens = world.spawn_actor(imu_bp, carla.Transform(), attach_to=pojazd)

imu_data = {'imu': None}

def imu_callback(data, data_dict):
    data_dict['imu'] = {
        'gyro': data.gyroscope,
        'accel': data.accelerometer,
        'compass': data.compass
    }
    
imu_sens.listen(lambda data: imu_callback(data, imu_data))

In [ ]:

en = 0.1
I = 0
e_pop = 0 

"""
Implementacja regulatora PID wykorzystywanego w tescie Łosia 

Wykorzystywana do sterowania prędkością pojazdu w celu ustalenia jej na zadanej wartości

y - obecna prędkość pojazdu

limMin - ograniczenie dolne sterowania pojazdem (minimalna wartość jaką możę zwrócić funkcja PID, min = 0)

limMax - ograniczenie górny sterowana pojazdem (maksymalna wartość jaką może zwórcić funkcja PID, maks = 1)

zadana - zadana prędkość którą chcemy osiągnąć

Nastawy otrzymane przy użyciu metody Ziglera-Nicholsa

"""   
def pid(y, limMin, limMax, zadana):
    global I
    global e_pop
    h = 1
    kp = 32.2        # 0.35 : 0.35 : 50 : 54 
    Ti = 0.3333   # 4    : 0.1  : 0.01 : 0.2
    Td = 0.0777 # 0.5  : 0.5  : 0    : 0.5

    e = zadana - y
    P = kp*e
    
    I = I + (h*kp/Ti) * e

    D = (Td/h) * (e - e_pop) 
    
    e_pop = e

    sterowanie = P + I + D
    if sterowanie>limMax:
        sterowanie = limMax
        I = I - (h*kp/Ti) * e
    elif sterowanie<limMin:
        sterowanie = limMin
        I = I - (h*kp/Ti) * e
    return sterowanie



In [14]:
#Stworzenie pliku CSV
csv_filename = 'predkosci.csv'

# Nagłówki dla pliku CSV
csv_headers = ['Czas', 'Predkosc (km/h)']

# Otwórz plik CSV w trybie zapisu i zapisz nagłówki
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(csv_headers)

In [ ]:
#Główny program
flaga = 0 # flaga na 0 przyspieszamy do zadanej prędkości flaga na 1 hamujemy
skret = 0
hamulec = 0
pedalGazu = 1
font = cv2.FONT_HERSHEY_SIMPLEX
org2 = (30, 50)
fontScale = 0.5
color = (255, 255, 255)
grubosc = 1
ymax = 0

cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])
status = 1

# Telemetria
#pojazd.show_debug_telemetry()

while True:
    # Carla Tick
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        break
    image = camera_data['image']
    v = pojazd.get_velocity() 
   
    s = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    pit = pojazd.get_transform().rotation
    pit = pit.pitch
    
    # Zbieranie polozenia pojazdu
    
    transformacja = pojazd.get_transform()

    x = transformacja.location.x
    y = transformacja.location.y
    z = transformacja.location.z
    
    if y > ymax:
        ymax = y

    current_time = time.time()
   
    # Zapisz do pliku CSV
    with open(csv_filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([current_time, s ,pit,pedalGazu,hamulec,x,y,z])
    image = cv2.putText(image,'Predkosc: '+str(int(s))+' km/h',org2, 
                        font, fontScale,color,grubosc,cv2.LINE_AA)
  

    gaz  = pid(s, 0.25, 1, 100)
    pojazd.apply_control(carla.VehicleControl(throttle=gaz, steer=skret))
    
        
    cv2.imshow('RGB Camera',image)



print(ymax)
cv2.destroyAllWindows()
camera.stop()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()

In [ ]:
#Pojazdy
#'vehicle.lincoln.mkz_2020'
#'vehicle.mercedes.coupe_2020'
#'vehicle.mercedes.sprinter'
#'vehicle.ford.ambulance'
#'vehicle.'volkswagen.t2_2021'
#'vehicle.mitsubishi.fusorosa'
#'vehicle.tesla.cybertruck'
#'vehicle.mini.cooper_s_2021'

